In [1]:
import requests
import base64
import json
import re
import hashlib

# import mysql.connector
from dotenv import load_dotenv
import os


In [6]:
def campos(fields):
    cad = ''
    for x in fields.keys():
        cad += x + ' = ' + fields[x] + ', '
    return cad[:-2]

def leeactzl(user, repo_name, path_to_file):
    json_url ='https://api.github.com/repos/{}/{}/contents/{}'.format(user, repo_name,
                                                                      path_to_file)
    response = requests.get(json_url) #get data from json file located at specified URL 

    if response.status_code == requests.codes.ok:
        jsonResponse = response.json()  # the response is a JSON
        #the JSON is encoded in base 64, hence decode it
        content = base64.b64decode(jsonResponse['content'])
        #convert the byte stream to string
        jsonString = content.decode('utf-8')
        return json.loads(jsonString)
 
    else:
        return 'Content was not found.'
    
def calcquerys(finalJson):
    
    querys = []
    for registro in finalJson:
        #if registro['firma'] == hashlib.sha1(registro["instruccion"].encode('utf-8')).hexdigest():
        dt_query = json.loads(registro['instruccion'])

        quetmp = []
        for y in dt_query.keys():
            if dt_query[y]['op'] == 'UPDATE':
                query  = str(dt_query[y]['op']) + ' ' + y + ' SET ' + campos(dt_query[y]['fields'])
                query += ' WHERE '
            elif dt_query[y]['op'] == 'INSERT INTO':
                query  = query = str(dt_query[y]['op']) + ' ' + y
                query += ' ' + str(tuple(list(dt_query[y]['fields'].keys())))+ ' VALUES '
                query +=  str(tuple(list(dt_query[y]['fields'].values())))
                query  = query.replace("'", "")
                filtro_update = 'coduser = ' + dt_query[y]['fields']['coduser']
            quetmp.append(query)

        quetmp = [q + filtro_update if 'UPDATE' in q else q for q in quetmp]

        querys.extend(quetmp)

            #break # <--== Pilas quitar en produccion

    
    patron = re.compile('[0-9]{4}-[0-9]{2}-[0-9]{2}')

    for i, query in enumerate(querys):
        for r in patron.findall(querys[i]):
            querys[i] = querys[i].replace(r, "'" + r + "'")

    return querys

def conexion(query):
    try:
        cnx = mysql.connector.connect(user=sql_id, password=sql_pw, host='127.0.0.1', database='clientes')
        cursor = cnx.cursor()
        seleccion = (query)
        cursor.execute(seleccion)
        cnx.commit()
        return 'ok'
    except:
        cnx = mysql.connector.connect(user=sql_id, password=sql_pw, host='127.0.0.1', database='clientes')
        cursor = cnx.cursor()
        seleccion = (query)
        cursor.execute(seleccion)
        return list(cursor)

In [3]:
user = 'sistelca'
repo_name = 'desechosSolidos'
path_to_file = 'data_actzl.json'

finalJson = leeactzl(user, repo_name, path_to_file)


In [34]:
instrucions =  {'update': ['set', 'where'],
                 'insert into': ['(', ') values'],
                 'delete from': ['where', 'todo']
               } 

for registro in finalJson:
    #if registro['firma'] == hashlib.sha1(registro["instruccion"].encode('utf-8')).hexdigest():
    dt_query = json.loads(registro['instruccion'])
    regq = ''
    for y in dt_query.keys():
        z = dt_query[y]
        asi = z['op']
        gna = instrucions[asi][0]
        fl  = instrucions[asi][1]
        regq += z['op'] +' ' + y + ' ' + gna + z['set'] + fl + z['filtro']
    print(regq)
    print(registro['firma'] == hashlib.sha1(regq.encode('utf-8')).hexdigest())
    



update datos_red set fech_pag='2021-05-05' where coduser='314'insert into histori_pags (coduser, fech_pag, cant_bf, fech_venc, anfit, cod) values (314, '2021-05-05', 200000, '2021-06-05', '192.168.45.221', '7d02d2bac5f8f7d69829b31a603d750e')update datos_per set fech_ing='2021-05-05', fech_ven='2021-06-05' where coduser='314'
False
update datos_red set fech_pag='2021-05-05' where coduser='410'insert into histori_pags (coduser, fech_pag, cant_bf, fech_venc, anfit, cod) values (410, '2021-05-05', 200000, '2021-06-05', '192.168.45.221', '7d02d2bac5f8f7d69829b31a603d750e')update datos_per set fech_ing='2021-05-05', fech_ven='2021-06-05' where coduser='410'
False
update datos_red set fech_pag='2021-05-05' where coduser='347'insert into histori_pags (coduser, fech_pag, cant_bf, fech_venc, anfit, cod) values (347, '2021-05-05', 200000, '2021-06-05', '192.168.45.64', '')update datos_per set fech_ing='2021-05-05', fech_ven='2021-06-05' where coduser='347'
False
update datos_red set fech_pag='202

In [4]:
finalJson

[{'id': 1,
  'fecha': '2021-05-05 07:13:57',
  'instruccion': '{"datos_red":{"op":"update","set":" fech_pag=\'2021-05-05\' ","filtro":" coduser=\'314\'"},"histori_pags":{"op":"insert into","set":"coduser, fech_pag, cant_bf, fech_venc, anfit, cod","filtro":" (314, \'2021-05-05\', 200000, \'2021-06-05\', \'192.168.45.221\', \'7d02d2bac5f8f7d69829b31a603d750e\')"},"datos_per":{"op":"update","set":" fech_ing=\'2021-05-05\', fech_ven=\'2021-06-05\' ","filtro":" coduser=\'314\'"}}',
  'firma': '93b95c9ed5f0abdd1ad10b56d13632964d58905d'},
 {'id': 2,
  'fecha': '2021-05-05 08:08:24',
  'instruccion': '{"datos_red":{"op":"update","set":" fech_pag=\'2021-05-05\' ","filtro":" coduser=\'410\'"},"histori_pags":{"op":"insert into","set":"coduser, fech_pag, cant_bf, fech_venc, anfit, cod","filtro":" (410, \'2021-05-05\', 200000, \'2021-06-05\', \'192.168.45.221\', \'7d02d2bac5f8f7d69829b31a603d750e\')"},"datos_per":{"op":"update","set":" fech_ing=\'2021-05-05\', fech_ven=\'2021-06-05\' ","filtro":" c

In [40]:
query1 = "update datos_red set fech_pag = '2021-03-21' where coduser = 347"
query2 = "insert into histori_pags (coduser, fech_pag, cant_Bf, fech_venc) values (347, '2021-04-05', 100000, '2021-04-21')"
query3 = "update datos_per set fech_ing = '2021-04-05', fech_ven = '2021-04-21' where coduser = 347"


qs = query1+query2+query3

In [41]:
qa = "update datos_red set fech_pag = '2021-03-21' where coduser = 347insert into histori_pags (coduser, fech_pag, cant_bf, fech_venc) values (347, '2021-04-05', 100000, '2021-04-21')update datos_per set fech_ing = '2021-04-05', fech_ven = '2021-04-21' where coduser = 347"

hashlib.sha1(qs.encode('utf-8')).hexdigest(), hashlib.sha1(qa.encode('utf-8')).hexdigest(), 

('7ebab62959c2e17a2ce7caa83c7a8812a0905896',
 '591a350d06438f0e13c4dbf65cec13164bd6a822')

In [49]:
insert into histori_pags (coduser, fech_pag, cant_bf, fech_venc) values (347, '2021-04-05', 100000, '2021-04-21')
insert into histori_pags (coduser, fech_pag, cant_Bf, fech_venc) values (347, '2021-04-05', 100000, '2021-04-21')

266 266


In [52]:
tm = qa.replace(query1, '')
tm = tm.replace(query3, '')
tm


"insert into histori_pags (coduser, fech_pag, cant_bf, fech_venc) values (347, '2021-04-05', 100000, '2021-04-21')"

In [56]:
tm == query2

False